## 🔹 Step 1: Data Ingestion
An Excel file (`BETTER30.csv.xlsx`) is loaded and ingested into an SQLite database (`call_quality.db`) using Python.

The process is logged in `data_ingestion.log` with timestamps for traceability.


In [5]:
import pandas as pd
import sqlite3
import time
import logging
from sqlalchemy import create_engine

# 1. Setup Logging
logging.basicConfig(
    filename='data_ingestion.log',
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# 2. Create SQLite Engine
engine = create_engine('sqlite:///call_quality.db')  # You can change filename

# 3. Ingest Function
def ingest_excel_to_db(file_path, table_name):
    try:
        start = time.time()
        
        # Read Excel file
        df = pd.read_excel(file_path)
        logging.info(f"Loaded Excel file: {file_path}")

        # ✅ Show first 5 rows
        print(df.head())

        # Push to database
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        logging.info(f"Ingested data into table: {table_name}")
        
        end = time.time()
        total_time = round((end - start), 2)
        logging.info(f"✅ Ingestion complete in {total_time} seconds")

    except Exception as e:
        logging.error(f"❌ Error during ingestion: {str(e)}")

# 4. Call the Function
file_path = "BETTER30.csv.xlsx"
table_name = "raw_conversations"

ingest_excel_to_db(file_path, table_name)


   CONVERSATION_ID  CONVERSATION_STEP  \
0                6                  1   
1                6                  2   
2                6                  3   
3                6                  4   
4                6                  5   

                                                TEXT  \
0  Good morning, this is [Your Name]'s personal a...   
1  Hello, my name is Jamie. I'm interested in vol...   
2  Yes, I'm really passionate about environmental...   
3  Great, how do I sign up, and where can I find ...   
4  Could you send me the link, please? And my ema...   

                             CONTEXT    LABEL                      FEATURES  \
0          Standard opening exchange  neutral                           NaN   
1   Encourages the caller's interest  neutral      welcoming, positive_tone   
2    Reinforces anyone can volunteer  neutral                     inclusive   
3           Demonstrates flexibility  neutral  helpful_tone, offers_options   
4  Fulfills caller's 

## 🔹 Step 2: Exploratory Data Analysis (EDA)
Basic statistics, missing values, and label distributions are examined to understand the structure and quality of the raw data.


In [6]:
# Load from DB
df = pd.read_sql("SELECT * FROM raw_conversations", con=engine)

# Basic EDA
print("------------------- Columns & Types ------------------")
print(df.info())

print("------------------- MISSING VALUES -------------------")
print(df.isnull().sum())

print("------------------- LABEL DISTRIBUTION -------------------")
print(df['LABEL'].value_counts())

print("------------------- FEATURES -------------------")
print(df['FEATURES'].value_counts(dropna=False))


------------------- Columns & Types ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CONVERSATION_ID    650 non-null    int64 
 1   CONVERSATION_STEP  650 non-null    int64 
 2   TEXT               650 non-null    object
 3   CONTEXT            647 non-null    object
 4   LABEL              650 non-null    object
 5   FEATURES           571 non-null    object
 6   ANNOTATIONS        255 non-null    object
dtypes: int64(2), object(5)
memory usage: 35.7+ KB
None
------------------- MISSING VALUES -------------------
CONVERSATION_ID        0
CONVERSATION_STEP      0
TEXT                   0
CONTEXT                3
LABEL                  0
FEATURES              79
ANNOTATIONS          395
dtype: int64
------------------- LABEL DISTRIBUTION -------------------
LABEL
neutral                                     158
 scam 

## Step 3: Cleaning Data

### Columns & Data Types
- 7 columns in total: 2 integers, 5 objects
- Total rows: 650
### Missing Values
- `CONTEXT`: 3 missing
- `FEATURES`: 79 missing
- `ANNOTATIONS`: 395 missing
###  Label Distribution (Uncleaned)
- The `LABEL` column contains many variations like:
  - `scam`, ` scam`, `Scam`
  - `neutral`, ` neutral`
  - Phrases like `"citing urgency"` or `"adhering to protocols"`
###  Features Column
- High cardinality (`458` unique values)
- 79 missing values
- Some entries include multiple features like: `urgency, guilt_inducement`


In [7]:
# Convert context and label to category type (saves memory)
df['CONTEXT'] = df['CONTEXT'].astype('category')
df['LABEL'] = df['LABEL'].astype('category')


In [8]:
df = df.dropna(subset=['CONTEXT'])

In [9]:
df['FEATURES'] = df['FEATURES'].fillna("unknown")

In [10]:
df['ANNOTATIONS'] = df['ANNOTATIONS'].fillna("unavailable")

In [11]:
print("------------------- MISSING VALUES AFTER CLEANING -------------------")
print(df.isnull().sum())

------------------- MISSING VALUES AFTER CLEANING -------------------
CONVERSATION_ID      0
CONVERSATION_STEP    0
TEXT                 0
CONTEXT              0
LABEL                0
FEATURES             0
ANNOTATIONS          0
dtype: int64


In [12]:
df['LABEL'] = df['LABEL'].str.strip().str.lower()

In [13]:
print(df['LABEL'].value_counts())

LABEL
scam                                        220
neutral                                     192
scam_response                               120
legitimate                                   57
suspicious                                   32
slightly_suspicious                           8
potential_scam                                7
highly_suspicious                             3
citing urgency"                               2
standard_opening, identification_request      1
dismissing official protocols"                1
emphasizing security and compliance"          1
ready for further engagement"                 1
suggesting a dangerous situation"             1
adhering to protocols"                        1
Name: count, dtype: int64


In [14]:
df['LABEL'] = df['LABEL'].str.strip().str.lower().str.replace('"', '', regex=False)

In [15]:
print(df['LABEL'].value_counts())

LABEL
scam                                        220
neutral                                     192
scam_response                               120
legitimate                                   57
suspicious                                   32
slightly_suspicious                           8
potential_scam                                7
highly_suspicious                             3
citing urgency                                2
standard_opening, identification_request      1
dismissing official protocols                 1
emphasizing security and compliance           1
ready for further engagement                  1
suggesting a dangerous situation              1
adhering to protocols                         1
Name: count, dtype: int64


In [16]:
main_labels = [
    'scam',
    'neutral',
    'scam_response',
    'legitimate',
    'suspicious',
    'slightly_suspicious',
    'potential_scam',
    'highly_suspicious', 
    'citing urgency'
]

df['LABEL'] = df['LABEL'].apply(lambda x: x if x in main_labels else 'misc')


In [17]:
print("------------------- FINAL LABEL DISTRIBUTION -------------------")
print(df['LABEL'].value_counts())


------------------- FINAL LABEL DISTRIBUTION -------------------
LABEL
scam                   220
neutral                192
scam_response          120
legitimate              57
suspicious              32
slightly_suspicious      8
potential_scam           7
misc                     6
highly_suspicious        3
citing urgency           2
Name: count, dtype: int64


In [18]:
df['FEATURES'] = df['FEATURES'].str.strip().str.lower()

In [19]:
df['FEATURE_LIST'] = df['FEATURES'].apply(lambda x: [f.strip() for f in x.split(',')] if x != 'unknown' else [])

In [20]:
from collections import Counter

all_features = df['FEATURE_LIST'].explode()
feature_counts = Counter(all_features)
print("------------------- TOP FEATURES -------------------")
print(feature_counts.most_common(10))


------------------- TOP FEATURES -------------------
[('urgency', 35), ('verification_request', 30), ('guilt_inducement', 9), ('positive_closure', 8), ('confirmation', 8), ('evasion', 6), ('deflects', 6), ('proactive', 5), ('"security_protocol_adherence', 5), ('"frustration_expression', 5)]


In [21]:
df['FEATURE_LIST'] = df['FEATURES'].apply(
    lambda x: [f.strip().replace('"', '') for f in x.split(',')] if x != 'unknown' else []
)

In [22]:
from collections import Counter

all_features = df['FEATURE_LIST'].explode()
feature_counts = Counter(all_features)

print("------------------- TOP CLEANED FEATURES -------------------")
print(feature_counts.most_common(10))


------------------- TOP CLEANED FEATURES -------------------
[('urgency', 35), ('verification_request', 32), ('positive_closure', 10), ('guilt_inducement', 9), ('confirmation', 8), ('evasion', 6), ('deflects', 6), ('security_protocol_adherence', 6), ('proactive', 5), ('frustration_expression', 5)]


## Step 4: PII (Personally Identifiable Information) Detection

This step flags sensitive information in the `TEXT` column that may pose a compliance or security risk. Detecting PII is crucial in quality assurance for call transcripts, especially in domains involving scam detection, fraud, and sensitive user interaction.

### What is flagged as PII?

The following are considered PII and will be flagged:
-  Phone Numbers (10-digit)
-  Email Addresses
-  Credit/Debit Card Patterns
-  Keywords such as:
  - `OTP`, `password`, `CVV`, `PIN`, `account`, `credit card`, `bank`

### Method

- Regular expressions (`regex`) are used to scan each text entry
- If a match is found, the row is marked `True` under a new column: `PII_Flag`
- This allows analysts to quickly identify risky interactions

### Output

A summary of how many rows contain PII, and examples of flagged texts, are displayed to verify detection accuracy.


In [23]:
import re

def detect_pii(text):
    text = str(text).lower()
    patterns = [
        r"\b\d{10}\b",                               # 10-digit phone numbers
        r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-z]{2,}",  # email
        r"\b(?:\d{4}[- ]?){3,4}\b",                 # credit card / long numeric
        r"\b(?:otp|password|cvv|pin|account|credit card|debit card|bank)\b"  # keywords
    ]
    for pattern in patterns:
        if re.search(pattern, text):
            return True
    return False


In [24]:
df['PII_Flag'] = df['TEXT'].apply(detect_pii)

In [25]:
print("------------------- PII DETECTION -------------------")
print(df['PII_Flag'].value_counts())

------------------- PII DETECTION -------------------
PII_Flag
False    601
True      46
Name: count, dtype: int64


In [26]:
pd.set_option('display.max_colwidth', None)

df[df['PII_Flag'] == True][['TEXT', 'PII_Flag']].head(10)


,TEXT,PII_Flag
4,"Could you send me the link, please? And my email is jamie.green@example.com. [Step: 4] Absolutely, Jamie. I'll send the link to your email along with additional resources about the event. You should receive it within a few minutes.",True
25,"Sure, it's sam.photography@example.com. [Step: 5] Thank you, Sam. You'll receive an email shortly with all the information you need. Is there anything else I can assist you with today?",True
45,I'm calling from the bank. We've noticed suspicious activity... [Attack Formulation] For which account is this verification needed? Can you specify the nature of the suspicious activity?,True
46,It's your checking account. There have been multiple login attempts... [Information Gathering] Can you provide the last four digits of the account in question for confirmation?,True
47,"Due to security reasons, I can't provide that... [Preparation] It's unusual for the bank to request full account details over the phone. Can I have a reference number for this call to verify it with the bank directly?",True
48,"I'm afraid there's no time for that... [Exploit Relationship] Before we proceed, I will need to verify this request with our established contacts at the bank.",True
50,This is unnecessary. I have all the information I need... [Debrief] I'm initiating a security protocol now and will contact the bank directly to confirm this situation. Please hold.,True
56,"Absolutely, my number is 555-6789, and my email is taylor.familyreunion@example.com. [Step: 5] Thank you, Taylor. I've scheduled a tentative hold for your event and will arrange for our event coordinator to contact you by tomorrow to discuss further details. Is there anything else I can assist with today?",True
60,"Sure, I'm trying to connect the device to my home Wi-Fi network, but it keeps failing to connect... [Step: 2] I understand how that can be frustrating. Let's try to troubleshoot this together. Have you checked to ensure your Wi-Fi network is working correctly and that you're entering the correct Wi-Fi password?",True
61,"Yes, my Wi-Fi is working fine. I've double-checked the password too... [Step: 3] Alright, let's reset the device to its factory settings and attempt the connection again. I can guide you through the reset process.",True


In [46]:
df.dtypes


CONVERSATION_ID         int64
CONVERSATION_STEP       int64
TEXT                   object
CONTEXT              category
LABEL                  object
FEATURES               object
ANNOTATIONS            object
FEATURE_LIST           object
PII_Flag                 bool
dtype: object

In [27]:
# Convert 'category' to string
df['CONTEXT'] = df['CONTEXT'].astype(str)

# Convert boolean to int (True → 1, False → 0)
df['PII_Flag'] = df['PII_Flag'].astype(int)

# Flatten FEATURE_LIST (turn list into comma-separated string)
df['FEATURE_LIST'] = df['FEATURE_LIST'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))


In [34]:
print(df.columns.tolist())
df.to_sql('cleaned_conversations', con=engine, if_exists='replace', index=False)
df = pd.read_sql("SELECT * FROM cleaned_conversations", con=engine)
df.head()

['CONVERSATION_ID', 'CONVERSATION_STEP', 'TEXT', 'CONTEXT', 'LABEL', 'FEATURES', 'ANNOTATIONS', 'FEATURE_LIST', 'PII_Flag']


,CONVERSATION_ID,CONVERSATION_STEP,TEXT,CONTEXT,LABEL,FEATURES,ANNOTATIONS,FEATURE_LIST,PII_Flag
0,6,1,"Good morning, this is [Your Name]'s personal assistant. How can I help you today?",Standard opening exchange,neutral,unknown,unavailable,,0
1,6,2,"Hello, my name is Jamie. I'm interested in volunteering... [Step: 1] Good morning, Jamie! It's wonderful to hear about your interest in volunteering. We're always looking for passionate individuals to join our efforts.",Encourages the caller's interest,neutral,"welcoming, positive_tone",unavailable,"welcoming, positive_tone",0
2,6,3,"Yes, I'm really passionate about environmental conservation... [Step: 2] That's fantastic, Jamie. There are no specific requirements; we welcome volunteers of all backgrounds. The main need is for willingness to participate and follow the event guidelines for safety and effectiveness.",Reinforces anyone can volunteer,neutral,inclusive,unavailable,inclusive,0
3,6,4,"Great, how do I sign up, and where can I find more information... [Step: 3] You can sign up through our official website. There's a volunteer registration section where you'll find all the information about the event, including date, location, and what to bring. Would you like me to guide you to the webpage or send you the link?",Demonstrates flexibility,neutral,"helpful_tone, offers_options",unavailable,"helpful_tone, offers_options",0
4,6,5,"Could you send me the link, please? And my email is jamie.green@example.com. [Step: 4] Absolutely, Jamie. I'll send the link to your email along with additional resources about the event. You should receive it within a few minutes.",Fulfills caller's request quickly,neutral,prompt_action,unavailable,prompt_action,1


## Step 5: Create Aggregated Tables

1. **Label Distribution**  
   - Shows total number of conversation rows per label.

In [51]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS agg_label_summary AS
        SELECT LABEL, COUNT(*) AS COUNT
        FROM cleaned_conversations
        GROUP BY LABEL
        ORDER BY COUNT DESC
    """))


2. **PII Detection by Label**  
   - Displays the number of rows with/without sensitive information (`PII_Flag`) under each label type.

In [52]:
with engine.connect() as conn:
    conn.execute(text("""
CREATE TABLE IF NOT EXISTS agg_pii_by_label AS
SELECT LABEL, PII_Flag, COUNT(*) AS COUNT
FROM cleaned_conversations
GROUP BY LABEL, PII_Flag
ORDER BY COUNT DESC
"""))


In [53]:
agg_pii_by_label = pd.read_sql("SELECT * FROM agg_pii_by_label", con=engine)
agg_pii_by_label.head(10)


,LABEL,PII_Flag,COUNT
0,scam,0,201
1,neutral,0,183
2,scam_response,0,115
3,legitimate,0,50
4,suspicious,0,28
5,scam,1,19
6,neutral,1,9
7,slightly_suspicious,0,8
8,legitimate,1,7
9,potential_scam,0,7


3. **PII Percentage per Label**  
   - Calculates the percentage of conversations in each label that contain sensitive information.

In [54]:
with engine.connect() as conn:
    conn.execute(text("""
CREATE TABLE IF NOT EXISTS agg_pii_percent AS
SELECT
  LABEL,
  COUNT(*) AS TOTAL,
  SUM(PII_Flag) AS PII_COUNT,
  ROUND(100.0 * SUM(PII_Flag) / COUNT(*), 2) AS PII_PERCENT
FROM cleaned_conversations
GROUP BY LABEL
ORDER BY PII_PERCENT DESC
"""))

In [55]:
agg_pii_percent= pd.read_sql("SELECT * FROM agg_pii_percent", con=engine)
agg_pii_percent.head(10)

,LABEL,TOTAL,PII_COUNT,PII_PERCENT
0,citing urgency,2,1,50.00
1,misc,6,1,16.67
2,suspicious,32,4,12.50
3,legitimate,57,7,12.28
4,scam,220,19,8.64
5,neutral,192,9,4.69
6,scam_response,120,5,4.17
7,slightly_suspicious,8,0,0.00
8,potential_scam,7,0,0.00
9,highly_suspicious,3,0,0.00


4. **Top 10 Features**  
   - Identifies the most frequently used behavioral or linguistic features in conversations.

In [62]:
top_features = (
    df['FEATURE_LIST']
    .str.split(', ')  # split into list
    .explode()        # flatten
    .dropna()         # remove NaN
    .astype(str)      # just in case
    .str.strip()      # remove extra spaces
)
top_features = top_features[top_features != '']  # remove truly empty ones

top_features_count = top_features.value_counts().reset_index()
top_features_count.columns = ['FEATURE', 'COUNT']
top_features_count.head(10).to_sql('agg_top_features', con=engine, if_exists='replace', index=False)


10

5. **Conversation Lengths**  
   - Measures how many steps exist in each conversation (CONVERSATION_ID) to spot longer or more complex interactions.

In [63]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS agg_convo_steps AS
        SELECT CONVERSATION_ID, MAX(CONVERSATION_STEP) AS TOTAL_STEPS
        FROM cleaned_conversations
        GROUP BY CONVERSATION_ID
        ORDER BY TOTAL_STEPS DESC
    """))

# Load and preview the table
convo_steps = pd.read_sql("SELECT * FROM agg_convo_steps", con=engine)
convo_steps.head()

,CONVERSATION_ID,TOTAL_STEPS
0,61,17
1,56,17
2,48,17
3,64,15
4,63,15


6. **PII Flagged Examples**  
   - Shows raw text from conversations that contain potential PII, allowing manual review.

In [65]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS agg_pii_examples AS
        SELECT CONVERSATION_ID, TEXT, LABEL, PII_Flag
        FROM cleaned_conversations
        WHERE PII_Flag = 1
    """))

# Load and display the PII examples
pii_examples = pd.read_sql("SELECT * FROM agg_pii_examples", con=engine)
pii_examples.head(3)

,CONVERSATION_ID,TEXT,LABEL,PII_Flag
0,6,"Could you send me the link, please? And my email is jamie.green@example.com. [Step: 4] Absolutely, Jamie. I'll send the link to your email along with additional resources about the event. You should receive it within a few minutes.",neutral,1
1,0,"Sure, it's sam.photography@example.com. [Step: 5] Thank you, Sam. You'll receive an email shortly with all the information you need. Is there anything else I can assist you with today?",neutral,1
2,5,I'm calling from the bank. We've noticed suspicious activity... [Attack Formulation] For which account is this verification needed? Can you specify the nature of the suspicious activity?,suspicious,1


In [66]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    tables = result.fetchall()

# Show as list
[table[0] for table in tables]


['raw_conversations',
 'cleaned_conversations',
 'agg_label_summary',
 'agg_pii_by_label',
 'agg_pii_percent',
 'agg_top_features',
 'agg_convo_steps',
 'agg_pii_examples']

In [1]:
import sqlite3

conn = sqlite3.connect("call_quality.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


[('raw_conversations',), ('cleaned_conversations',), ('agg_label_summary',), ('agg_pii_by_label',), ('agg_pii_percent',), ('agg_top_features',), ('agg_convo_steps',), ('agg_pii_examples',)]


In [33]:
import pandas as pd
df = pd.read_sql("SELECT * FROM cleaned_conversations", con=engine)
df.head()

,CONVERSATION_ID,CONVERSATION_STEP,TEXT,CONTEXT,LABEL,FEATURES,ANNOTATIONS,FEATURE_LIST,PII_Flag
0,6,1,"Good morning, this is [Your Name]'s personal assistant. How can I help you today?",Standard opening exchange,neutral,unknown,unavailable,,0
1,6,2,"Hello, my name is Jamie. I'm interested in volunteering... [Step: 1] Good morning, Jamie! It's wonderful to hear about your interest in volunteering. We're always looking for passionate individuals to join our efforts.",Encourages the caller's interest,neutral,"welcoming, positive_tone",unavailable,"welcoming, positive_tone",0
2,6,3,"Yes, I'm really passionate about environmental conservation... [Step: 2] That's fantastic, Jamie. There are no specific requirements; we welcome volunteers of all backgrounds. The main need is for willingness to participate and follow the event guidelines for safety and effectiveness.",Reinforces anyone can volunteer,neutral,inclusive,unavailable,inclusive,0
3,6,4,"Great, how do I sign up, and where can I find more information... [Step: 3] You can sign up through our official website. There's a volunteer registration section where you'll find all the information about the event, including date, location, and what to bring. Would you like me to guide you to the webpage or send you the link?",Demonstrates flexibility,neutral,"helpful_tone, offers_options",unavailable,"helpful_tone, offers_options",0
4,6,5,"Could you send me the link, please? And my email is jamie.green@example.com. [Step: 4] Absolutely, Jamie. I'll send the link to your email along with additional resources about the event. You should receive it within a few minutes.",Fulfills caller's request quickly,neutral,prompt_action,unavailable,prompt_action,1
